In [6]:
x <- 1
x

   1


In [7]:
for (i in 1:4) {
    y[i] <- i
}
y

   [ 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]


In [9]:
for (i in 1:4) {
    z[i] ~ dnNormal(0,1)
}
z

   [ -0.395, 0.809, 2.443, -0.208, -0.714, 1.025, -0.804, 0.800, 0.023, 0.637]


In [ ]:
for (i in 1:4) {
    for (j in 1:4) {
        w[i][j] ~ dnBeta()
    }
}